In [29]:
from Bio import AlignIO
import os
import subprocess
from Bio import AlignIO
from Bio.Align import MultipleSeqAlignment
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq

In [30]:
# Modify these for the comparision you would want to see
# for the first part please select 16S, 23S or bactRNAseP
# Caution they should exactly match as above otherwise it wouldn't work
aes1 = ["bactRNAseP", "63_0"]
aes2 = ["16S",  "19_0"]

## Execute below to run cmcompare

In [36]:
def combine_alignments(align1, align2, map1, map2, suffix1="align1", suffix2="align2"):
    """
    Combine two alignments using provided mapping indices.
    
    Parameters:
    align1, align2: MultipleSeqAlignment objects
    map1, map2: Lists of indices for mapping positions between alignments
    
    Returns:
    MultipleSeqAlignment object containing the combined alignment
    """
    # Create a mapping dictionary
    position_map = {m1: m2 for m1, m2 in zip(map1, map2)}
    
    # Find the maximum position in both mappings
    max_pos = max(len(map1), len(map2))
    
    # Initialize combined sequences for each record in align1
    combined_records = []
    
    for record in align1:
        # Create a new sequence with gaps
        new_seq = ['-'] * max_pos
        
        # Fill in positions from align1
        for i, pos in enumerate(map1):
            if pos < len(record.seq):
                new_seq[i] = record.seq[pos-1]
                
                
        # Create new SeqRecord
        new_record = SeqRecord(
            Seq(''.join(new_seq)),
            id=record.id + "|" + suffix1,
            name=record.name,
            description=f"Combined alignment from {suffix1}"
        )
        combined_records.append(new_record)
    
    # Add sequences from align2
    for record in align2:
        new_seq = ['-'] * max_pos
        
        # Fill in positions from align2
        for i, pos in enumerate(map2):
            if pos < len(record.seq):
                new_seq[i] = record.seq[pos-1]
                
        # Create new SeqRecord
        new_record = SeqRecord(
            Seq(''.join(new_seq)),
            id=record.id + "|" + suffix2,
            name=record.name,
            description=f"Combined alignment from {suffix2}"
        )
        combined_records.append(new_record)
    
    # Create new MultipleSeqAlignment
    combined_alignment = MultipleSeqAlignment(combined_records)
    
    return combined_alignment

In [37]:
cm_dir = "/home/sumon/workspace/git_repos/aes_db/data/all_cms_noSS_1p"

def compare_models(args):
    name1, name2 = args
    cm1 = os.path.join(cm_dir, name1)
    cm2 = os.path.join(cm_dir, name2)
    result = subprocess.run(["./executables/cmcompare", cm1, cm2], 
                            capture_output=True, text=True)
    std_out = result.stdout
    return std_out


In [38]:
unit_mapping = {"16S": "S", "23S": "L", "bactRNAseP": "P"}

cm1 = f"{unit_mapping[aes1[0]]}{aes1[1]}.cm"
cm2 = f"{unit_mapping[aes2[0]]}{aes2[1]}.cm"

op = compare_models((cm1, cm2))
op_list = list(filter(None, op.split(" ")))

# create the alignment based on cmcompare result
exec("st_idx1 = " + op_list[-2])
exec("st_idx2 = " + op_list[-1])

st1 = f"/home/sumon/workspace/git_repos/aes_db/cm_builder_storage/data/{aes1[0]}/stockholms/aes_{aes1[1]}.stockholm"
st2 = f"/home/sumon/workspace/git_repos/aes_db/cm_builder_storage/data/{aes2[0]}/stockholms/aes_{aes2[1]}.stockholm"
align1 = AlignIO.read(st1, "stockholm")
align2 = AlignIO.read(st2, "stockholm")

suffix1 = cm1.replace(".cm", "")
suffix2 = cm2.replace(".cm", "")
combined_msa = combine_alignments(align1, align2, st_idx1, st_idx2, suffix1=suffix1, suffix2=suffix2)


with open(f"tmp/{suffix1}__{suffix2}.fasta", "w") as output_handle:
    AlignIO.write(combined_msa, output_handle, "fasta")
    

with open(f"tmp/{suffix1}__{suffix2}.cmcompare.out", "w") as output_handle:
    output_handle.writelines("\n".join(op_list))

## Execute below to see results

In [39]:
print(f"{': cmcompare output :':-^120}")
print()
print("\n".join(op_list))
print()
print(120*"-")

print(f"File written: tmp/{suffix1}__{suffix2}.fasta")

--------------------------------------------------: cmcompare output :--------------------------------------------------

/home/sumon/workspace/git_repos/aes_db/data/all_cms_noSS_1p/P63_0.cm
/home/sumon/workspace/git_repos/aes_db/data/all_cms_noSS_1p/S19_0.cm
20.856
14.428
GGUGGGGUAAGGGCCUACCAAUUUUUUUUUUGACGGGGGCGGCUA
...................,.......................,.
.....................,,,,,,,,,,,,,,,,,,,.....
[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,42,43,44]
[10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,31,32,33,34,35,36]


------------------------------------------------------------------------------------------------------------------------
File written: tmp/P63_0__S19_0.fasta


## Extra stuffs ignore

In [8]:
aes1 = ["16S",  "19_0"]
aes2 = ["bactRNAseP", "63_0"]

In [9]:
st1 = f"/home/sumon/workspace/git_repos/aes_db/cm_builder_storage/data/{aes1[0]}/stockholms/aes_{aes1[1]}.stockholm"
st2 = f"/home/sumon/workspace/git_repos/aes_db/cm_builder_storage/data/{aes2[0]}/stockholms/aes_{aes2[1]}.stockholm"
align1 = AlignIO.read(st1, "stockholm")
align2 = AlignIO.read(st2, "stockholm")


with open(f"tmp/{suffix1}.fasta", "w") as output_handle:
    AlignIO.write(align1, output_handle, "fasta")
    
with open(f"tmp/{suffix2}.fasta", "w") as output_handle:
    AlignIO.write(align2, output_handle, "fasta")
